# Toronto Neighborhood - Segmenting & Clustering

## Part 01 - Scapping Data from wikipedia page

In [1]:
! pip install bs4

import requests
from bs4 import BeautifulSoup

     |████████████████████████████████| 122kB 19.1MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text

soup = BeautifulSoup(data,'html.parser')

In [3]:
dataTable = list()
table = soup.find('table')
for row in table.findAll('td'):
    if row.span.text == 'Not assigned':
        pass
    else:
        cell = {}
        cell['PostCode'] = row.p.text[:3]
        cell['Borough'] = row.span.text.split('(')[0]
        cell['Neighborhood'] = row.span.text.split('(')[1].strip(')').replace(' /',',')
        dataTable.append(cell)


In [4]:
import pandas as pd

df = pd.DataFrame(dataTable)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
df['Neighborhood'].shape

(103,)

## Part 02 - Extracting Coordinates for the Boroughs

In [6]:
dfCoordinates = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
dfCoordinates.rename(columns={'Postal Code':'PostCode'}, inplace=True)
df = pd.merge(df,dfCoordinates,on='PostCode')
df.head()

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Part 03 - Clustering Neighborhood

In [7]:
df2 = df[df['Borough'].str.contains('Toronto',regex=False)]
df2.head()

,PostCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [8]:
from sklearn.cluster import KMeans

k = 5
df3 = df2[['Latitude','Longitude']]
KMmodel = KMeans(n_clusters = k).fit(df3)
KMmodel.labels_

array([1, 1, 1, 3, 1, 1, 4, 1, 2, 3, 1, 4, 3, 1, 4, 3, 1, 3, 0, 0, 0, 0,
       2, 0, 4, 2, 0, 4, 2, 0, 4, 0, 1, 1, 1, 1, 1, 1, 3], dtype=int32)

In [9]:
df2.insert(0,'ClusterLabels',KMmodel.labels_)
df2.head()

,ClusterLabels,PostCode,Borough,Neighborhood,Latitude,Longitude
2,1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,1,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,3,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [14]:
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=12)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Neighborhood'], df2['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters